# Processing Excel Spreedsheets for Payment Locations
1. Process each file
2. Run the Geocode
3. Truncate/Load to existing 
4. Still need republish?

# ToDo
~~1. Add in  Google Link Column~~  
~~2.Process each field with .title() method~~  
~~3. Process Phone Numbers~~  
~~4. Process Zip code to only have first 5~~  


~~Add in correct Google Link~~  
Symbol for each type  
    * This will require a decision to add in a column that is unused
    * How many columns would we need because there are dups and there are 3 different types
        * Cash, RTP, Money Gram
Geocode  
Find duplicates
    * Sometimes it could be Cash, RTP, and Money Gram 
    * Have seperate columns that have the capability for each location?
    
Write functions that are repitive

In [23]:
#import arcpy
import pandas as pd
import os
import geocoder

In [24]:
# Read all the files in directory
for excel_file in os.listdir("paymentlocations"):
    if excel_file.endswith(".xls"):
        print(excel_file)
    if excel_file.endswith("xlsx"):
        print(excel_file)

ACE Cash Express.xlsx
Fidelity Express.xls
MoneyGram.xlsx
RTP's.xlsx


# Ace Cash Express Processing Requirements
### use .tile() on all fields (cells)
**1. ACE Cash Express**  
* Symbol = white  
* Columns to keep - 
    1. Center Name = LocationName (Alias: Location Name)
    2. Address
    3. City
    4. ST
    5. Zip = ZipCode
    6. Phone - Check for 1 in beginning; Phone will add parentheses and dash(-) where appropriate - (972)292-5000
    7. Longitude
    8. Latitude  
    * Symbol
    * Cash
    * RTP
    * Money Gram

In [25]:
# Read in excel sheet and take only desired columns
# Rename the column names
columnName = ["LocationName", "Address", "City", "ST", "ZipCode", "Phone", "Longitude", "Latitude"]
# ace = pd.read_excel("paymentlocations/ACE Cash Express.xlsx")
ace = pd.read_excel("paymentlocations/ACE Cash Express.xlsx", sheet_name="Sheet1", usecols="B:G, V:W", names=columnName)
ace.head()

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude
0,ACE CASH EXPRESS #0315,3117 AL LIPSCOMB WAY,DALLAS,TX,75215,2144214894,-96.768468,32.774702
1,ACE CASH EXPRESS 0318,3029 SOUTH FRWY,FORT WORTH,TX,76104,8179219722,-97.320144,32.707227
2,ACE CASH EXPRESS 0320,201 W MAIN ST,GRAND PRAIRIE,TX,75050,9722627919,-97.005206,32.745328
3,ACE CASH EXPRESS 0321,500 N GALLOWAY AVE,MESQUITE,TX,75149,9722891201,-96.598901,32.771057
4,ACE CASH EXPRESS 0323,3418 OAK LAWN AVE,DALLAS,TX,75219,2145202274,-96.806379,32.811101


In [26]:
# Phone will add parentheses and dash(-) where appropriate - (972)292-5000
# type(ace["Phone"]) # 
phone_number = ace["Phone"]
phone_number = phone_number.astype(str)
# Check if phone number has a 1
for phone in phone_number:
    if phone[0] == "1":
        print("There is a 1")

In [27]:
#print(phone_number.head())
first_3 = phone_number.str.slice(stop=3)
#print(first_3.head())
mid_3 = phone_number.str.slice(start=3,stop=6)
#print(next_3.head())
last_4 = phone_number.str.slice(start=6)
#print(last_4)
phone_number = "(" + first_3 + ") " + mid_3 + "-" + last_4
phone_number.head()

0    (214) 421-4894
1    (817) 921-9722
2    (972) 262-7919
3    (972) 289-1201
4    (214) 520-2274
Name: Phone, dtype: object

In [28]:
# Process each column to have each new word start with a capital letter
ace["LocationName"] = ace["LocationName"].apply(lambda x: x.title())
ace["Address"] = ace["Address"].apply(lambda x: x.title())
ace["City"] = ace["City"].apply(lambda x: x.title())
ace["Phone"] = phone_number
ace.head()

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude
0,Ace Cash Express #0315,3117 Al Lipscomb Way,Dallas,TX,75215,(214) 421-4894,-96.768468,32.774702
1,Ace Cash Express 0318,3029 South Frwy,Fort Worth,TX,76104,(817) 921-9722,-97.320144,32.707227
2,Ace Cash Express 0320,201 W Main St,Grand Prairie,TX,75050,(972) 262-7919,-97.005206,32.745328
3,Ace Cash Express 0321,500 N Galloway Ave,Mesquite,TX,75149,(972) 289-1201,-96.598901,32.771057
4,Ace Cash Express 0323,3418 Oak Lawn Ave,Dallas,TX,75219,(214) 520-2274,-96.806379,32.811101


# Fidelity Processing Requirements
### use .tile() on all column names or just pull in each column to appropriate column
### use .tile() on all fields (cells)
**2. Fidelity Express**
* Symbol = white 
* Columns to keep
    1. Location Name = LocationName (Alias: Location Name)
    2. Address
    3. City
    4. ST
    5. Zip = ZipCode - take the first 5
    6. Loc Phone = Phone - Check for 1 in beginning; Phone will add parentheses and dash(-) where appropriate - (972)292-5000
    7. Longitude - taking an existing column that is used in the xls file
    8. Latitude - taking an existing column that is used in the xls file

In [65]:
# Read in excel sheet and take only desired columns
# Rename the column names
columnName = ["LocationName", "Address", "City", "ST", "ZipCode", "Phone", "Longitude", "Latitude"]
#fidelity = pd.read_excel("paymentlocations/Fidelity Express.xls")
fidelity = pd.read_excel("paymentlocations/Fidelity Express.xls", sheet_name="Sheet1", usecols="A:E, G:I", names=columnName)
fidelity.head()

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude
0,Polo`s Grocery & Laundry,621 E Oakwood St,Tyler,TX,757025943,9035926402,6AM-11PM,6AM-11PM
1,SUPER STORE,912 DIVISION ST,ATCHISON,KS,660020000,9133673548,6AM-11PM,6AM-11PM
2,STAR GAS,788 SPRUCE ST,LEAVENWORTH,KS,660480000,9136826026,6AM-11PM,6AM-11PM
3,FAV TRIP,1300 S 4TH ST,LEAVENWORTH,KS,660480000,9136828747,24HRS,24HRS
4,HOMETOWN STORE,111 N BROADWAY ST,LEAVENWORTH,KS,660481845,9136828330,6AM-11PM,6AM-11PM


In [30]:
# Phone will add parentheses and dash(-) where appropriate - (972)292-5000
phone_number = fidelity["Phone"]
phone_number = phone_number.astype(str)
# Check if phone number has a 1
for phone in phone_number:
    if phone[0] == "1":
        print("There is a 1")

In [31]:
#print(phone_number.head())
first_3 = phone_number.str.slice(stop=3)
#print(first_3.head())
mid_3 = phone_number.str.slice(start=3,stop=6)
#print(next_3.head())
last_4 = phone_number.str.slice(start=6)
#print(last_4)
phone_number = "(" + first_3 + ") " + mid_3 + "-" + last_4
phone_number.head()

0    (903) 592-6402
1    (913) 367-3548
2    (913) 682-6026
3    (913) 682-8747
4    (913) 682-8330
Name: Phone, dtype: object

In [32]:
# Process zip codes to only have first 5
zip_code = fidelity["ZipCode"]
zip_code = zip_code.astype(str)
zip_code = zip_code.str.slice(stop=5)
zip_code.head()

0    75702
1    66002
2    66048
3    66048
4    66048
Name: ZipCode, dtype: object

In [74]:
# Geocode
# Get the address - sample: g = geocoder.google("453 Booth Street, Ottawa ON")
# Store the geocoded long| Lat into a Series
address = fidelity["Address"] + ", " + fidelity["City"] + " " + fidelity["ST"]
address
# try an flist with fidelity["Longitude"]
for single_address in address:
    lng = geocoder.arcgis(single_address).lng
    print(single_address, lng)
    # Iterate through each fidelity["Longitude"]
    print(fidelity["Longitude"])# = lng # I think...

621 E Oakwood St, Tyler TX -95.29416891618096
0       6AM-11PM
1       6AM-11PM
2       6AM-11PM
3          24HRS
4       6AM-11PM
5       10AM-9PM
6       9AM-10PM
7        9AM-9PM
8       6AM-10PM
9       10AM-8AM
10      6AM-12AM
11         24HRS
12       8AM-5PM
13      6AM-10PM
14         24HRS
15         24HRS
16      6AM-11PM
17      10AM-7PM
18         24HRS
19       9AM-8PM
20      6AM-11PM
21      10AM-6PM
22      10AM-6PM
23       8AM-8PM
24       9AM-5PM
25       9AM-5PM
26         24HRS
27       8AM-8PM
28      5AM-11PM
29         24HRS
          ...   
2070     8AM-8PM
2071     8AM-7PM
2072     8AM-6PM
2073     8AM-7PM
2074    10AM-6PM
2075    8AM-10PM
2076    5AM-12AM
2077    8AM-10PM
2078     8AM-8PM
2079    7AM-11PM
2080    6AM-10PM
2081    8AM-10PM
2082    5AM-12AM
2083    11AM-6PM
2084     8AM-5PM
2085     7AM-7PM
2086    8AM-10PM
2087    8AM-10PM
2088     9AM-6PM
2089     8AM-9PM
2090     8AM-9PM
2091     8AM-7PM
2092     8AM-7PM
2093     9AM-7PM
2094     9AM-6PM
20

12885 S MURLEN RD., OLATHE KS -94.77953617439007
0       6AM-11PM
1       6AM-11PM
2       6AM-11PM
3          24HRS
4       6AM-11PM
5       10AM-9PM
6       9AM-10PM
7        9AM-9PM
8       6AM-10PM
9       10AM-8AM
10      6AM-12AM
11         24HRS
12       8AM-5PM
13      6AM-10PM
14         24HRS
15         24HRS
16      6AM-11PM
17      10AM-7PM
18         24HRS
19       9AM-8PM
20      6AM-11PM
21      10AM-6PM
22      10AM-6PM
23       8AM-8PM
24       9AM-5PM
25       9AM-5PM
26         24HRS
27       8AM-8PM
28      5AM-11PM
29         24HRS
          ...   
2070     8AM-8PM
2071     8AM-7PM
2072     8AM-6PM
2073     8AM-7PM
2074    10AM-6PM
2075    8AM-10PM
2076    5AM-12AM
2077    8AM-10PM
2078     8AM-8PM
2079    7AM-11PM
2080    6AM-10PM
2081    8AM-10PM
2082    5AM-12AM
2083    11AM-6PM
2084     8AM-5PM
2085     7AM-7PM
2086    8AM-10PM
2087    8AM-10PM
2088     9AM-6PM
2089     8AM-9PM
2090     8AM-9PM
2091     8AM-7PM
2092     8AM-7PM
2093     9AM-7PM
2094     9AM-6PM

914 S 55TH ST, KANSAS CITY KS -94.70527437610345
0       6AM-11PM
1       6AM-11PM
2       6AM-11PM
3          24HRS
4       6AM-11PM
5       10AM-9PM
6       9AM-10PM
7        9AM-9PM
8       6AM-10PM
9       10AM-8AM
10      6AM-12AM
11         24HRS
12       8AM-5PM
13      6AM-10PM
14         24HRS
15         24HRS
16      6AM-11PM
17      10AM-7PM
18         24HRS
19       9AM-8PM
20      6AM-11PM
21      10AM-6PM
22      10AM-6PM
23       8AM-8PM
24       9AM-5PM
25       9AM-5PM
26         24HRS
27       8AM-8PM
28      5AM-11PM
29         24HRS
          ...   
2070     8AM-8PM
2071     8AM-7PM
2072     8AM-6PM
2073     8AM-7PM
2074    10AM-6PM
2075    8AM-10PM
2076    5AM-12AM
2077    8AM-10PM
2078     8AM-8PM
2079    7AM-11PM
2080    6AM-10PM
2081    8AM-10PM
2082    5AM-12AM
2083    11AM-6PM
2084     8AM-5PM
2085     7AM-7PM
2086    8AM-10PM
2087    8AM-10PM
2088     9AM-6PM
2089     8AM-9PM
2090     8AM-9PM
2091     8AM-7PM
2092     8AM-7PM
2093     9AM-7PM
2094     9AM-6PM

600 SE QUINCY, TOPEKA KS -95.67194697859107
0       6AM-11PM
1       6AM-11PM
2       6AM-11PM
3          24HRS
4       6AM-11PM
5       10AM-9PM
6       9AM-10PM
7        9AM-9PM
8       6AM-10PM
9       10AM-8AM
10      6AM-12AM
11         24HRS
12       8AM-5PM
13      6AM-10PM
14         24HRS
15         24HRS
16      6AM-11PM
17      10AM-7PM
18         24HRS
19       9AM-8PM
20      6AM-11PM
21      10AM-6PM
22      10AM-6PM
23       8AM-8PM
24       9AM-5PM
25       9AM-5PM
26         24HRS
27       8AM-8PM
28      5AM-11PM
29         24HRS
          ...   
2070     8AM-8PM
2071     8AM-7PM
2072     8AM-6PM
2073     8AM-7PM
2074    10AM-6PM
2075    8AM-10PM
2076    5AM-12AM
2077    8AM-10PM
2078     8AM-8PM
2079    7AM-11PM
2080    6AM-10PM
2081    8AM-10PM
2082    5AM-12AM
2083    11AM-6PM
2084     8AM-5PM
2085     7AM-7PM
2086    8AM-10PM
2087    8AM-10PM
2088     9AM-6PM
2089     8AM-9PM
2090     8AM-9PM
2091     8AM-7PM
2092     8AM-7PM
2093     9AM-7PM
2094     9AM-6PM
2095

KeyboardInterrupt: 

In [33]:
# Process each column to have each new word start with a capital letter
fidelity["LocationName"] = fidelity["LocationName"].apply(lambda x: x.title())
fidelity["Address"] = fidelity["Address"].apply(lambda x: x.title())
fidelity["City"] = fidelity["City"].apply(lambda x: x.title())
fidelity["Phone"] = phone_number
fidelity["ZipCode"] = zip_code
fidelity.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,Polo`S Grocery & Laundry,621 E Oakwood St,Tyler,TX,75702,(903) 592-6402
1,Super Store,912 Division St,Atchison,KS,66002,(913) 367-3548
2,Star Gas,788 Spruce St,Leavenworth,KS,66048,(913) 682-6026
3,Fav Trip,1300 S 4Th St,Leavenworth,KS,66048,(913) 682-8747
4,Hometown Store,111 N Broadway St,Leavenworth,KS,66048,(913) 682-8330


In [34]:
# Phone will add parentheses and dash(-) where appropriate - (972)292-5000
phone_number = fidelity["Phone"]
phone_number = phone_number.astype(str)
# Check if phone number has a 1
for phone in phone_number:
    if phone[0] == "1":
        print("There is a 1")

In [35]:
#print(phone_number.head())
first_3 = phone_number.str.slice(stop=3)
#print(first_3.head())
mid_3 = phone_number.str.slice(start=3,stop=6)
#print(next_3.head())
last_4 = phone_number.str.slice(start=6)
#print(last_4)
phone_number = "(" + first_3 + ") " + mid_3 + "-" + last_4
phone_number.head()

0    ((90) 3) -592-6402
1    ((91) 3) -367-3548
2    ((91) 3) -682-6026
3    ((91) 3) -682-8747
4    ((91) 3) -682-8330
Name: Phone, dtype: object

In [36]:
# Process zip codes to only have first 5
zip_code = fidelity["ZipCode"]
zip_code = zip_code.astype(str)
zip_code = zip_code.str.slice(stop=5)
zip_code.head()

0    75702
1    66002
2    66048
3    66048
4    66048
Name: ZipCode, dtype: object

In [37]:
# Process each column to have each new word start with a capital letter
fidelity["LocationName"] = fidelity["LocationName"].apply(lambda x: x.title())
fidelity["Address"] = fidelity["Address"].apply(lambda x: x.title())
fidelity["City"] = fidelity["City"].apply(lambda x: x.title())
fidelity["Phone"] = phone_number
fidelity["ZipCode"] = zip_code
fidelity.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,Polo`S Grocery & Laundry,621 E Oakwood St,Tyler,TX,75702,((90) 3) -592-6402
1,Super Store,912 Division St,Atchison,KS,66002,((91) 3) -367-3548
2,Star Gas,788 Spruce St,Leavenworth,KS,66048,((91) 3) -682-6026
3,Fav Trip,1300 S 4Th St,Leavenworth,KS,66048,((91) 3) -682-8747
4,Hometown Store,111 N Broadway St,Leavenworth,KS,66048,((91) 3) -682-8330


## Money Gram Processing Requirements
### use .tile() on all fields (cells)
**3. Money Gram**
* Symbol = white 
* Columns to keep
    1. Display Name = LocationName (Alias: Location Name)
    2. Phyiscal Address Line 1 = Address
    3. Phyiscal Address City = City
    4. Phyiscal Address State/Province Code = ST
    5. Zip Code = ZipCode - take the first 5
    6. Phone Number = Phone - Check for 1 in beginning; Phone will add parentheses and dash(-) where appropriate - (972)292-5000

In [38]:
# Read in excel sheet and take only desired columns
# Rename the column names
columnName = ["LocationName", "Address", "City", "ST", "ZipCode", "Phone"]
#moneygram = pd.read_excel("paymentlocations/MoneyGram.xls")
moneygram = pd.read_excel("paymentlocations/MoneyGram.xlsx", usecols="B:E, G:H", names=columnName, skiprows=0)
moneygram.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,Y GAS & OIL - #0001,6257 E STATE HIGHWAY 107,EDINBURG,TX,785427292,9563816464
1,DOC'S BEVERAGE,1103 E ENNIS AVE,ENNIS,TX,751194342,9728759451
2,TOP GROCERIES,33 W MAIN ST,HUMBLE,TX,773384308,2815483359
3,AMIGO SUPERMARKET - #0001,6362 WINDSWEPT LN,HOUSTON,TX,770577210,7139149288
4,FAST CASH CHECK SERVICE,1314 N COUNTY RD W,ODESSA,TX,797633440,4323358211


In [39]:
# Phone will add parentheses and dash(-) where appropriate - (972)292-5000
phone_number = moneygram["Phone"]
phone_number = phone_number.astype(str)
# Check if phone number has a 1
for phone in phone_number:
    if phone[0] == "1":
        print("There is a 1")

In [40]:
#print(phone_number.head())
first_3 = phone_number.str.slice(stop=3)
#print(first_3.head())
mid_3 = phone_number.str.slice(start=3,stop=6)
#print(next_3.head())
last_4 = phone_number.str.slice(start=6)
#print(last_4)
phone_number = "(" + first_3 + ") " + mid_3 + "-" + last_4
phone_number.head()

0    (956) 381-6464
1    (972) 875-9451
2    (281) 548-3359
3    (713) 914-9288
4    (432) 335-8211
Name: Phone, dtype: object

In [41]:
# Process zip codes to only have first 5
zip_code = moneygram["ZipCode"]
zip_code = zip_code.astype(str)
zip_code = zip_code.str.slice(stop=5)
zip_code.head()

0    78542
1    75119
2    77338
3    77057
4    79763
Name: ZipCode, dtype: object

In [42]:
# Process each column to have each new word start with a capital letter
moneygram["LocationName"] = moneygram["LocationName"].apply(lambda x: x.title())
moneygram["Address"] = moneygram["Address"].apply(lambda x: x.title())
moneygram["City"] = moneygram["City"].apply(lambda x: x.title())
moneygram["Phone"] = phone_number
moneygram["ZipCode"] = zip_code
moneygram.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,Y Gas & Oil - #0001,6257 E State Highway 107,Edinburg,TX,78542,(956) 381-6464
1,Doc'S Beverage,1103 E Ennis Ave,Ennis,TX,75119,(972) 875-9451
2,Top Groceries,33 W Main St,Humble,TX,77338,(281) 548-3359
3,Amigo Supermarket - #0001,6362 Windswept Ln,Houston,TX,77057,(713) 914-9288
4,Fast Cash Check Service,1314 N County Rd W,Odessa,TX,79763,(432) 335-8211


# RTP Processing Requirements
### use .tile() on all fields (cells)
**4. RTP**
* Symbol = blue
* Columns to keep
    1. DESCRIPTION = LocationName (Alias: Location Name)
    2. ADDRESS = Address
    3. CITY = City
    4. STATE = ST
    5. ZIP = ZipCode
    6. PHONE = Phone - Check for 1 in beginning; remove the dash(-); Phone will add parentheses and dash(-) where appropriate - (972)292-5000

In [43]:
# Read in xlsx as an object
columnName = ["LocationName", "Address", "City", "ST", "ZipCode", "Phone"]
#rtp = pd.read_excel("paymentlocations/RTP's.xlsx")
rtp = pd.read_excel("paymentlocations/RTP's.xlsx", sheet_name="Sheet1", usecols="A, C:G", names=columnName)
rtp.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,ALBERTSONS - #4279,833 NE ALSBURY BLVD,BURLESON,TX,76028,817-447-9106
1,ALBERTSONS - #4290,480 NORTHWEST PARKWAY,AZLE,TX,76020,817-270-1116
2,ALBERTSONS - #4176,225 E. SPRING ST,WEATHERFORD,TX,76086,817-594-9390
3,CITY OF BURLESON,141 W RENFRO ST,BURLESON,TX,76028,817-426-9600
4,CITY OF GARLAND,217 N. FIFTH ST,GARLAND,TX,75040,972-205-2646


In [44]:
# Phone will add parentheses and dash(-) where appropriate - (972)292-5000
phone_number = rtp["Phone"]
phone_number = phone_number.astype(str)
# Check if phone number has a 1
for phone in phone_number:
    if phone[0] == "1":
        print("There is a 1")

In [45]:
#print(phone_number.head())
phone_number = phone_number.str.replace("-", "")
first_3 = phone_number.str.slice(stop=3)
#print(first_3.head())
mid_3 = phone_number.str.slice(start=3,stop=6)
#print(next_3.head())
last_4 = phone_number.str.slice(start=6)
#print(last_4)
phone_number = "(" + first_3 + ") " + mid_3 + "-" + last_4
phone_number.head()

0    (817) 447-9106
1    (817) 270-1116
2    (817) 594-9390
3    (817) 426-9600
4    (972) 205-2646
Name: Phone, dtype: object

In [46]:
# Process each column to have each new word start with a capital letter
rtp["LocationName"] = rtp["LocationName"].apply(lambda x: x.title())
rtp["Address"] = rtp["Address"].apply(lambda x: x.title())
rtp["City"] = rtp["City"].apply(lambda x: x.title())
rtp["Phone"] = phone_number
rtp.head()

,LocationName,Address,City,ST,ZipCode,Phone
0,Albertsons - #4279,833 Ne Alsbury Blvd,Burleson,TX,76028,(817) 447-9106
1,Albertsons - #4290,480 Northwest Parkway,Azle,TX,76020,(817) 270-1116
2,Albertsons - #4176,225 E. Spring St,Weatherford,TX,76086,(817) 594-9390
3,City Of Burleson,141 W Renfro St,Burleson,TX,76028,(817) 426-9600
4,City Of Garland,217 N. Fifth St,Garland,TX,75040,(972) 205-2646


# Empty Dataframe to append each excel file
1. LocationName (Alias: Location Name)
2. Address
3. City
4. ST
5. ZipCode (Alias: Zip Code)
6. Phone
7. Longitude
8. Latitude
9. GoogleLink (Alias: Google Link)
10. Symbol

In [47]:
# Create the output datafrome to append each excel file
columnName=["LocationName", "Address", "City", "ST", "ZipCode", "Phone", "Longitude", "Latitude", "GoogleLink", "Symbol"]
add_columns = pd.DataFrame(columns=columnName)
add_columns

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude,GoogleLink,Symbol


In [48]:
# Add all the excel turned dataframes into 1 dataframe
# sort = Sorts the columns in alphabetical order
output = pd.concat([add_columns, ace, fidelity, moneygram, rtp], sort=False)

# Duplicate issues
#output.duplicated("Address")

In [51]:
# Add in Google Link
#output["ZipCode"] = output["ZipCode"].astype(str)
# https://www.google.com/maps/place/480 NORTHWEST PARKWAY, AZLE, TX. 76020
output["GoogleLink"] = "https://www.google.com/maps/search/" + output["Address"] + ", " + output["ST"] + " " + output["ZipCode"].astype(str)
output["GoogleLink"].head()
output

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude,GoogleLink,Symbol
0,Ace Cash Express #0315,3117 Al Lipscomb Way,Dallas,TX,75215,(214) 421-4894,-96.768468,32.774702,https://www.google.com/maps/search/3117 Al Lip...,NaN
1,Ace Cash Express 0318,3029 South Frwy,Fort Worth,TX,76104,(817) 921-9722,-97.320144,32.707227,https://www.google.com/maps/search/3029 South ...,NaN
2,Ace Cash Express 0320,201 W Main St,Grand Prairie,TX,75050,(972) 262-7919,-97.005206,32.745328,https://www.google.com/maps/search/201 W Main ...,NaN
3,Ace Cash Express 0321,500 N Galloway Ave,Mesquite,TX,75149,(972) 289-1201,-96.598901,32.771057,https://www.google.com/maps/search/500 N Gallo...,NaN
4,Ace Cash Express 0323,3418 Oak Lawn Ave,Dallas,TX,75219,(214) 520-2274,-96.806379,32.811101,https://www.google.com/maps/search/3418 Oak La...,NaN
5,Ace Cash Express 0324,5111 Greenville Ave,Dallas,TX,75206,(214) 696-2274,-96.770286,32.849112,https://www.google.com/maps/search/5111 Greenv...,NaN
6,Ace Cash Express 0328,1914 E Beltline Rd,Carrollton,TX,75006,(972) 416-2274,-96.889452,32.953430,https://www.google.com/maps/search/1914 E Belt...,NaN
7,Ace Cash Express 0331,2243 S Cooper St,Arlington,TX,76013,(817) 274-2274,-97.116262,32.707766,https://www.google.com/maps/search/2243 S Coop...,NaN
8,Ace Cash Express 0337,3658 N Belt Line Rd,Irving,TX,75062,(972) 252-1207,-96.991865,32.856015,https://www.google.com/maps/search/3658 N Belt...,NaN
9,Ace Cash Express 0341,1409 N Beltline Rd,Irving,TX,75061,(972) 986-2274,-96.994024,32.828911,https://www.google.com/maps/search/1409 N Belt...,NaN


In [52]:
# index=False is needed so there is no index that is added by Pandas
output.to_csv("paymentlocations/output.csv", index=False)
output.tail()

,LocationName,Address,City,ST,ZipCode,Phone,Longitude,Latitude,GoogleLink,Symbol
202,Tom Thumb - #2580,100 W. S.Lake Blvd,Southlake,TX,76092,(817) 416-5464,NaN,NaN,https://www.google.com/maps/search/100 W. S.La...,NaN
203,Tom Thumb - #1789,101 Trophy Lake Dr,Roanoke,TX,76262,(817) 490-6960,NaN,NaN,https://www.google.com/maps/search/101 Trophy ...,NaN
204,Tom Thumb - #3622,6377 Camp Bowie Blvd,Fort Worth,TX,76116,(817) 654-0256,NaN,NaN,https://www.google.com/maps/search/6377 Camp B...,NaN
205,Tom Thumb - #2574,3100 S. Hulen,Fort Worth,TX,76109,(817) 570-2950,NaN,NaN,https://www.google.com/maps/search/3100 S. Hul...,NaN
206,Tom Thumb - #3312,2400 W. 7Th St.,Fort Worth,TX,76107,(817) 302-1400,NaN,NaN,https://www.google.com/maps/search/2400 W. 7Th...,NaN
